<h2>Question of Analyze </h2>

"User like to purchase condo around ***Dusit District*** and the it must near ***Department Store*** and ***BTS(sky train)*** or ***MRT(subway)***

and if it posible it may near ***Fitness Center***"


In [1]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

import folium # plotting library
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [2]:

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


<h3>Define Function and Param</h3>

In [3]:
def getNeighborHoodLocation(address):
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    return location

In [4]:
def meetAllMainCriteria(inCateList):
    output = False
    tempMain = mainCriteria.copy()
    
    for i,criteria in enumerate(mainCriteria):
        checkCate = criteria
        #print("checkCate : ",checkCate)
        if any(s in checkCate for s in inCateList):
            tempMain.remove(checkCate)
    
    if(len(tempMain)==0):
        output = True
    
    #print(tempMain)
    return output
    

In [5]:
#seta = {'4bf58dd8d48988d1f6941735'}
#meetAllMainCriteria(seta)

In [6]:
def getListExtraCriteria(inCateList):
    
    setIntersec = set(inCateList).intersection(extraCriteria)
    output = []
    for i,a in enumerate(setIntersec):
        #output.append(a)
        output.append(list(allCategoryDF[allCategoryDF['id']==a]['name'])[0])
        
    return (output)

In [7]:
#Foursquare

CLIENT_ID = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
CLIENT_SECRET:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [8]:
def getAllCategory(categoryResp):
    categoryBranch = categoryResp['categories']
    idList = []
    nameList = []
    for i,node in enumerate(categoryBranch):
        idList.append(node['id'])
        nameList.append(node['name'])
        
        if('categories' in node.keys()):
            subDict = getAllCategory(node)
            
        idList.extend(subDict['id'])
        nameList.extend(subDict['name'])
    
    categoryDict = {'id':idList,'name':nameList}
    
    return categoryDict


In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        output = []
        output.append(categories_list[0]['id'])
        output.append(categories_list[0]['name'])
        return output

In [10]:
def getVenueNameCategoryType(venues):
    
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
    nearby_venues = nearby_venues.loc[:, filtered_columns]
    

    # filter the category for each row
    nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
    nearby_venues[['categoriesID','categories']] = pd.DataFrame(nearby_venues.categories.values.tolist(), index= nearby_venues.index)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    return nearby_venues


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            '100')
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],  
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category', 
                  'Venue CategoryID']
    
    return(nearby_venues)

<h2>Data Describe</h2>

result from check category through foursquare api

In [12]:
#get category

url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
url

results = requests.get(url).json()
json.dumps(results)[0:500]  #example print


'{"meta": {"code": 200, "requestId": "5be542d76a60712ec33a989c"}, "response": {"categories": [{"id": "4d4b7104d754a06370d81259", "name": "Arts & Entertainment", "pluralName": "Arts & Entertainment", "shortName": "Arts & Entertainment", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_", "suffix": ".png"}, "categories": [{"id": "56aa371be4b08b9a8d5734db", "name": "Amphitheater", "pluralName": "Amphitheaters", "shortName": "Amphitheater", "icon": {"prefix": "htt'

Response of category query is json

Try to parse it into DataFrame 

In [13]:
categoryDict = getAllCategory(results["response"])
allCategoryDF = pd.DataFrame(categoryDict)
allCategoryDF.head()

,id,name
0,4d4b7104d754a06370d81259,Arts & Entertainment
1,56aa371be4b08b9a8d5734db,Amphitheater
2,4fceea171983d5d06c3e9823,Aquarium
3,4bf58dd8d48988d1e1931735,Arcade
4,4bf58dd8d48988d1e2931735,Art Gallery


Looking for Category "***Condo***"

In [14]:
allCategoryDF[allCategoryDF['name'].str.contains("Condo") ]

,id,name
708,4d954b06a243a5684965b473,Residential Building (Apartment / Condo)


Found that category for condo is actully name "***Residential Building (Apartment / Condo)***" and id equals "***4d954b06a243a5684965b473***"

Looking for Category "***Department Store, Train Station, Metro Station and Fitness Center***"

In [15]:
#Department Store
allCategoryDF[allCategoryDF['name'].str.contains("Department Store") ]

,id,name
754,4bf58dd8d48988d1f6941735,Department Store


In [16]:
#Train Station
allCategoryDF[allCategoryDF['name'].str.contains("Train Station") ]

,id,name
930,4bf58dd8d48988d129951735,Train Station


In [17]:
#Metro Station
allCategoryDF[allCategoryDF['name'].str.contains("Metro Station") ]

,id,name
917,4bf58dd8d48988d1fd931735,Metro Station


In [18]:
#Fitness
allCategoryDF[allCategoryDF['name'].str.contains("Fitness") ]

,id,name
499,4bf58dd8d48988d175941735,Gym / Fitness Center


The Name of other category is ok but we will use ***Category ID*** because it unique, spaceless and only one value.

<h2>Initial Request Data</h2>

In [19]:
neighborhood = 'Dusit District Bangkok, Thailand'
neighborhoodRadius = 5000

From ***Data Describe*** I will change criteria from category name to category id

 
| Category Name        | Category ID           |
|:-------------:|:-------------:|
|Department Store | 4bf58dd8d48988d1f6941735  |
|Train Station | 4bf58dd8d48988d129951735  |
|Metro Station | 4bf58dd8d48988d1fd931735  |
|Fiteness Center | 4bf58dd8d48988d175941735  |

In [20]:
mainCriteria = ['4bf58dd8d48988d1f6941735', '4bf58dd8d48988d129951735|4bf58dd8d48988d1fd931735']
extraCriteria = ['4bf58dd8d48988d175941735']
condoRadius = 1000

<h2>Explore Condo in Neighborhood</h2>

In [21]:
neighborhoodLocation = getNeighborHoodLocation(neighborhood)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [22]:
#condo
condoCategoryID = '4d954b06a243a5684965b473'     
limit = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhoodLocation.latitude, neighborhoodLocation.longitude, VERSION, condoCategoryID, neighborhoodRadius, limit)
url

results = requests.get(url).json()
json.dumps(results)[0:500]  #example print

'{"meta": {"code": 200, "requestId": "5be542d8351e3d280e1fefe9"}, "response": {"venues": [{"id": "4cadf908bf70236a486704f9", "name": "Doctor Dormitory (\\u0e2b\\u0e2d\\u0e1e\\u0e31\\u0e01\\u0e41\\u0e1e\\u0e17\\u0e22\\u0e4c)", "location": {"address": "Ramathibodi Hospital", "lat": 13.76773115732634, "lng": 100.526239767615, "labeledLatLngs": [{"label": "display", "lat": 13.76773115732634, "lng": 100.526239767615}], "distance": 1576, "postalCode": "10400", "cc": "TH", "city": "\\u0e23\\u0e32\\u0e0a\\u0e40\\u0e17\\'

response from api is a json ,so I will parse it into dataframe

In [23]:
foundDF = getVenueNameCategoryType(results['response']['venues'])
foundDF.head()

,name,categories,lat,lng,categoriesID
0,Doctor Dormitory (หอพักแพทย์),College Residence Hall,13.767731,100.526240,4bf58dd8d48988d1a3941735
1,The Tree Rio Bang-Aor Station (เดอะทรีริโอ้ บา...,Residential Building (Apartment / Condo),13.800074,100.507650,4d954b06a243a5684965b473
2,Lumpini Place Rama 9-Ratchada (ลุมพินี เพลส พร...,Residential Building (Apartment / Condo),13.757777,100.571531,4d954b06a243a5684965b473
3,Villa Asoke (วิลล่า อโศก),Residential Building (Apartment / Condo),13.749121,100.561880,4d954b06a243a5684965b473
4,noble ratchada,Residential Building (Apartment / Condo),13.763941,100.569326,4d954b06a243a5684965b473


remove none condo result from the data frame

In [24]:
condoDF = foundDF[foundDF['categoriesID']== condoCategoryID]
condoDF.reset_index(drop=True, inplace=True)
condoDF.head()

,name,categories,lat,lng,categoriesID
0,The Tree Rio Bang-Aor Station (เดอะทรีริโอ้ บา...,Residential Building (Apartment / Condo),13.800074,100.507650,4d954b06a243a5684965b473
1,Lumpini Place Rama 9-Ratchada (ลุมพินี เพลส พร...,Residential Building (Apartment / Condo),13.757777,100.571531,4d954b06a243a5684965b473
2,Villa Asoke (วิลล่า อโศก),Residential Building (Apartment / Condo),13.749121,100.561880,4d954b06a243a5684965b473
3,noble ratchada,Residential Building (Apartment / Condo),13.763941,100.569326,4d954b06a243a5684965b473
4,333 Riverside,Residential Building (Apartment / Condo),13.807087,100.520187,4d954b06a243a5684965b473


<h3> Virtualize Condo Around Dusit District</h3>

In [25]:
# create map of Manhattan using latitude and longitude values
neighborhoodMap = folium.Map(location=[neighborhoodLocation.latitude, neighborhoodLocation.longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(condoDF['lat'], condoDF['lng'], condoDF['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(neighborhoodMap)  
    
neighborhoodMap

![img](https://github.com/newmint/Coursera_Capstone/blob/master/fig1.png "img")

If image do not display, please click [here](https://github.com/newmint/Coursera_Capstone/blob/master/fig1.png)

<h2>Explore venues around each Condo</h2>

In [26]:
condo_venues = getNearbyVenues(names=condoDF['name'], latitudes=condoDF['lat'], longitudes=condoDF['lng'], radius=1000)
condo_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue CategoryID
0,The Tree Rio Bang-Aor Station (เดอะทรีริโอ้ บา...,13.800074,100.50765,Baan Bang Aor (บ้านบางอ้อ),13.795761,100.507798,Café,4bf58dd8d48988d16d941735
1,The Tree Rio Bang-Aor Station (เดอะทรีริโอ้ บา...,13.800074,100.50765,Bang O Talae Phao (บางอ้อ ทะเลเผา),13.797381,100.508562,Seafood Restaurant,4bf58dd8d48988d1ce941735
2,The Tree Rio Bang-Aor Station (เดอะทรีริโอ้ บา...,13.800074,100.50765,ตลาดนัด จรัญสนิทวงศ์ 95/1,13.803872,100.511289,Flea Market,4bf58dd8d48988d1f7941735
3,The Tree Rio Bang-Aor Station (เดอะทรีริโอ้ บา...,13.800074,100.50765,icecream cafe ณ บางกรวย,13.806224,100.502430,Ice Cream Shop,4bf58dd8d48988d1c9941735
4,The Tree Rio Bang-Aor Station (เดอะทรีริโอ้ บา...,13.800074,100.50765,KFC Drive-Thru (เคเอฟซี ไดร์ฟทรู),13.794258,100.506616,Fried Chicken Joint,4d4ae6fc7a7b7dea34424761


<h3> Analyze Each Neighborhood</h3>

In [27]:
condo_venues.shape

(3566, 8)

we found 3566 venues near all condo in the list

Check each condo is near main criteria

In [29]:
condoMeetMainCriteria = []

for i,node in enumerate(condoDF['name']):
    condoName = node
    if(meetAllMainCriteria(set(condo_venues[condo_venues['Neighborhood']== condoName]['Venue CategoryID']))):
        condoMeetMainCriteria.append(condoName)
        
condoMeetMainCriteria

['The Address Chidlom',
 'Pathumwan Resort (ปทุมวันรีสอร์ท)',
 'Athenee Residence']

Select only meet main criteria condo from list and remove catogries and categoriesID

In [30]:
matchDF = condoDF[condoDF['name'].isin(condoMeetMainCriteria)].drop(['categories','categoriesID'], axis=1)
matchDF.reset_index(drop=True, inplace=True)
matchDF

,name,lat,lng
0,The Address Chidlom,13.747803,100.545584
1,Pathumwan Resort (ปทุมวันรีสอร์ท),13.754710,100.533998
2,Athenee Residence,13.741017,100.549427


Check condo again about near extra criteria

In [31]:
condoExtraCriteria = []
for i,node in enumerate(matchDF['name']):
    condoName = node
    condoExtraCriteria.append(getListExtraCriteria(set(condo_venues[condo_venues['Neighborhood']== condoName]['Venue CategoryID'])))

condoExtraCriteria

[['Gym / Fitness Center'], ['Gym / Fitness Center'], ['Gym / Fitness Center']]

add the extraCriteria meet to each condo

In [32]:
matchDF['Extra Option'] = condoExtraCriteria
matchDF

,name,lat,lng,Extra Option
0,The Address Chidlom,13.747803,100.545584,[Gym / Fitness Center]
1,Pathumwan Resort (ปทุมวันรีสอร์ท),13.754710,100.533998,[Gym / Fitness Center]
2,Athenee Residence,13.741017,100.549427,[Gym / Fitness Center]


<h3> Virtualize Match Criteria Condo </h3> 

In [33]:
# create map of Manhattan using latitude and longitude values
neighborhoodMap = folium.Map(location=[neighborhoodLocation.latitude, neighborhoodLocation.longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(matchDF['lat'], matchDF['lng'], matchDF['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(neighborhoodMap)  
    
neighborhoodMap

![img](https://github.com/newmint/Coursera_Capstone/blob/master/fig2.png "img")

If image do not display, please click [here](https://github.com/newmint/Coursera_Capstone/blob/master/fig2.png)

<h2> Summary </h2>

In [34]:
matchDF

,name,lat,lng,Extra Option
0,The Address Chidlom,13.747803,100.545584,[Gym / Fitness Center]
1,Pathumwan Resort (ปทุมวันรีสอร์ท),13.754710,100.533998,[Gym / Fitness Center]
2,Athenee Residence,13.741017,100.549427,[Gym / Fitness Center]


From the ***Result*** above tell we that

They have 3 condo match with the main criteria (near ***Department Store*** and ***BTS(sky train)*** or ***MRT(subway)***)

and all of them also near ***Fitness Center*** too.